<a href="https://colab.research.google.com/github/numberjuani/uniswap_v3_pool_comparison/blob/main/UniswapV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://thegraph.com/hosted-service/subgraph/uniswap/uniswap-v3

In [78]:
import requests
import pandas as pd
import json
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [79]:
query = """
{
  pools(first:1000,orderBy:totalValueLockedUSD, orderDirection:desc) {
    totalValueLockedUSD
    createdAtTimestamp
    feesUSD
    id
    token0 {
      name
    }
    token1 {
      name
    }
  }
}
"""

In [80]:
pools = requests.post("https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3",json={"query": query})
pools = json.loads(pools.content)
pools = pools['data']['pools']

In [81]:
pools = pd.DataFrame(pools)
pools['feesUSD'] = pools['feesUSD'].astype(float)
pools['totalValueLockedUSD'] = pools['totalValueLockedUSD'].astype(float)
pools['createdAtTimestamp'] = pd.to_datetime(pools['createdAtTimestamp'],unit='s')
pools['days_since_creation'] = pools['createdAtTimestamp'].apply(lambda x: (pd.Timestamp.now() - x).days)
pools = pools[pools['days_since_creation']>0].copy()
pools['average_daily_profit'] = pools['feesUSD']/pools['days_since_creation']
pools['unlevered_returns'] = 36500*(pools['average_daily_profit']/pools['totalValueLockedUSD'])
pools.sort_values(by='average_daily_profit',ascending=False,inplace=True)
pools

,totalValueLockedUSD,createdAtTimestamp,feesUSD,id,token0,token1,days_since_creation,average_daily_profit,unlevered_returns
4,3.257200e+08,2021-05-04 23:10:00,2.005046e+08,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,{'name': 'USD Coin'},{'name': 'Wrapped Ether'},574,349311.093743,39.143603
5,2.974318e+08,2021-05-05 21:42:11,1.554889e+08,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,{'name': 'USD Coin'},{'name': 'Wrapped Ether'},573,271359.282692,33.300449
9,1.251155e+08,2021-05-05 16:37:08,8.520036e+07,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,{'name': 'Wrapped Ether'},{'name': 'Tether USD'},574,148432.690851,43.302321
8,1.721103e+08,2021-05-04 20:09:34,5.822718e+07,0xcbcdf9626bc03e24f779434178a73a0b4bad62ed,{'name': 'Wrapped BTC'},{'name': 'Wrapped Ether'},574,101441.073514,21.512943
31,1.400323e+07,2022-03-17 11:32:15,1.637239e+07,0xac4b3dacb91461209ae9d41ec517c2b9cb1b7daf,{'name': 'ApeCoin'},{'name': 'Wrapped Ether'},258,63458.884263,165.408199
...,...,...,...,...,...,...,...,...,...
402,3.081992e+05,2021-07-24 19:04:18,0.000000e+00,0x3778f73e949c98817a1f96c58121323754b429da,{'name': 'Uniswap'},{'name': 'Ring.Exchange'},493,0.000000,0.000000
839,4.209450e+04,2022-02-26 01:29:16,0.000000e+00,0xdd475ebcd9c560cdfdea0461615240071fb12077,{'name': 'HEX'},{'name': 'Hedron'},277,0.000000,0.000000
411,2.970213e+05,2022-09-05 12:55:33,0.000000e+00,0xf6ed2390be39c783ae78893c91669eeb635d0429,{'name': 'HEX'},{'name': 'Maximus Decimus'},86,0.000000,0.000000
822,4.426771e+04,2022-09-21 23:20:35,0.000000e+00,0xe181f3d1cf81c94d770768d48d42d07a9e9819c7,{'name': 'HEX'},{'name': 'Maximus Base'},69,0.000000,0.000000
